In [1]:
import zarr
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
def build_mission_root(dataset_folder, mission):
    base = Path(dataset_folder) / mission / "data"
    mission_root = {}

    for d in sorted(base.iterdir()):
        if d.is_dir() and (d / ".zgroup").exists():
            try:
                g = zarr.open_group(store=zarr.storage.LocalStore(str(d)), mode="r")
                mission_root[d.name] = g
            except Exception as e:
                print(f"[skip] {d.name}: {e}")

    return mission_root

dataset_folder = Path("~/grand_tour_dataset/missions").expanduser()
mission = "2024-10-01-11-47-44"

mission_root = build_mission_root(dataset_folder, mission)

In [3]:
sensors = [
    "anymal_state_odometry",
    "anymal_state_state_estimator",
    "anymal_imu",
    "anymal_state_actuator",
    "anymal_command_twist",
    #"hdr_front",
    #"hdr_left",
    #"hdr_right"
]

In [4]:
# List each datafield per sensor and with its shape
def get_hz(x):
    timestamps = np.array(x)
    diffs = np.diff(timestamps)
    avg_dt = np.mean(diffs)
    frequency_hz = 1 / avg_dt
    return f"{frequency_hz:.2f}"

for sensor in sensors:
    timestamps = mission_root[sensor]["timestamp"]
    hz = get_hz(timestamps)
    print(f"{sensor} ({hz} Hz):\n")
    keys = list(mission_root[sensor].keys())
    keys.sort()
    for key in keys:
        print(f"-->     {key} {mission_root[sensor][key].shape}")
    print(f"------------------------------\n")
        

anymal_state_odometry (19.96 Hz):

-->     pose_cov (8663, 6, 6)
-->     pose_orien (8663, 4)
-->     pose_pos (8663, 3)
-->     sequence_id (8663,)
-->     timestamp (8663,)
-->     twist_ang (8663, 3)
-->     twist_cov (8663, 6, 6)
-->     twist_lin (8663, 3)
------------------------------

anymal_state_state_estimator (399.96 Hz):

-->     LF_FOOT_contact (173589,)
-->     LF_FOOT_friction_coef (173589,)
-->     LF_FOOT_normal (173589, 3)
-->     LF_FOOT_restitution_coef (173589,)
-->     LF_FOOT_state (173589,)
-->     LF_FOOT_wrench_force (173589, 3)
-->     LF_FOOT_wrench_torque (173589, 3)
-->     LH_FOOT_contact (173589,)
-->     LH_FOOT_friction_coef (173589,)
-->     LH_FOOT_normal (173589, 3)
-->     LH_FOOT_restitution_coef (173589,)
-->     LH_FOOT_state (173589,)
-->     LH_FOOT_wrench_force (173589, 3)
-->     LH_FOOT_wrench_torque (173589, 3)
-->     RF_FOOT_contact (173589,)
-->     RF_FOOT_friction_coef (173589,)
-->     RF_FOOT_normal (173589, 3)
-->     RF_FOOT_rest

In [5]:
TARGET_HZ = 50.0
DT = 1.0 / TARGET_HZ

def _to_np(x):
    return np.asarray(x[:]) if hasattr(x, '__getitem__') and not isinstance(x, np.ndarray) else np.asarray(x)

def _search_zoh_indices(src_ts, tgt_ts):
    """
    Vectorized zero order hold: for each target time, pick the last src index with src_ts <= tgt
    in english: find the index of the last source timestamp that happened at or before that time.
    Returns idx array (int) with -1 where no src sample exists yet
    """
    idx = np.searchsorted(src_ts, tgt_ts, side='right') - 1
    return idx

def _resample_group_zoh(group, tgt_ts, ts_key="timestamp", skip_keys=("timestamp","sequence_id")):
    """
    Resample all fields in a Zarr group to tgt_ts using ZOH.
    """
    out = {}
    src_ts = _to_np(group[ts_key])

    # Assure ascending timestamps
    if not np.all(src_ts[:-1] <= src_ts[1:]):
        order = np.argsort(src_ts)
        src_ts = src_ts[order]
        # Reorder all fields to keep arrays aligned
        for key in group.keys():
            if key in skip_keys: 
                continue
            arr = _to_np(group[key])
            out[key] = arr[order]  # temp store; we’ll overwrite after computing indices
        reordered = True
    else:
        reordered = False

    idx = _search_zoh_indices(src_ts, tgt_ts)  # -1 if tgt time is before first src sample
    # For each tgt time stamp, find which source timestamp (from og sensor) was the most recent reading that happened <= the tgt time
    # --> so idx are the row of the original sensor data to use for each new aligned time step

    # Build a safe index for gather; we’ll mask invalids later
    safe_idx = idx.copy()
    safe_idx[safe_idx < 0] = 0
    safe_idx[safe_idx >= len(src_ts)] = len(src_ts) - 1

    for key in group.keys():
        if key in skip_keys: 
            continue

        arr = _to_np(group[key]) if not (reordered and key in out) else out[key]
        # Gather
        res = arr[safe_idx]
        # Mask times before the first source sample (the -1s from _search_zoh_indices) as NaN 
        if res.dtype.kind in ('f',):  # floating types: use NaN
            res[idx < 0] = np.nan
        else:
            # For non-floats (ints, bools), you can choose a sentinel; here we keep first value.
            pass
        out[key] = res

    # Always return the resampled timestamps too (the grid)
    out["timestamp_50hz"] = tgt_ts
    return out

def _overlap_window(mission_root, sensors, ts_key="timestamp"):
    """Compute overlapping [start, end] across sensors to avoid extrapolation beyond last sample."""
    starts = []
    ends = []
    for s in sensors:
        ts = _to_np(mission_root[s][ts_key])
        starts.append(ts[0])
        ends.append(ts[-1])
    return max(starts), min(ends)

def build_50hz_grid(t_start, t_end):
    # Inclusive start, inclusive end if it lands exactly; otherwise stops before end
    n = int(np.floor((t_end - t_start) * TARGET_HZ)) + 1
    return (t_start + np.arange(n) * DT).astype(np.float64)

# main entrypoint
def align_mission_to_50hz(mission_root, sensors, ts_key="timestamp"):
    """
    Returns:
      {
        "t": np.ndarray [T],  # 50 Hz grid
        "sensors": {
            sensor_name: { field: np.ndarray[T, ...], "timestamp_50hz": np.ndarray[T] }
        }
      }
    """
    t0, t1 = _overlap_window(mission_root, sensors, ts_key=ts_key)
    tgt_ts = build_50hz_grid(t0, t1)

    aligned = {}
    for s in sensors:
        aligned[s] = _resample_group_zoh(mission_root[s], tgt_ts, ts_key=ts_key)

    return {"t": tgt_ts, "sensors": aligned}


aligned = align_mission_to_50hz(mission_root, sensors)

t = aligned["t"]  # 50 Hz timeline
base_lin_vel = aligned["sensors"]["anymal_state_state_estimator"]["twist_lin"]   
imu_ang_vel   = aligned["sensors"]["anymal_imu"]["ang_vel"]                      
cmd_linear    = aligned["sensors"]["anymal_command_twist"]["linear"]             


In [6]:
for sensor in sensors:
    print(f"{sensor}:")
    keys = list(aligned["sensors"][sensor].keys())
    keys.sort()
    for key in keys:
        print(f"-->    {key} {aligned["sensors"][sensor][key].shape} {type(aligned["sensors"][sensor][key])}")
    print(f"------------------------------\n")
        

anymal_state_odometry:
-->    pose_cov (18228, 6, 6) <class 'numpy.ndarray'>
-->    pose_orien (18228, 4) <class 'numpy.ndarray'>
-->    pose_pos (18228, 3) <class 'numpy.ndarray'>
-->    timestamp_50hz (18228,) <class 'numpy.ndarray'>
-->    twist_ang (18228, 3) <class 'numpy.ndarray'>
-->    twist_cov (18228, 6, 6) <class 'numpy.ndarray'>
-->    twist_lin (18228, 3) <class 'numpy.ndarray'>
------------------------------

anymal_state_state_estimator:
-->    LF_FOOT_contact (18228,) <class 'numpy.ndarray'>
-->    LF_FOOT_friction_coef (18228,) <class 'numpy.ndarray'>
-->    LF_FOOT_normal (18228, 3) <class 'numpy.ndarray'>
-->    LF_FOOT_restitution_coef (18228,) <class 'numpy.ndarray'>
-->    LF_FOOT_state (18228,) <class 'numpy.ndarray'>
-->    LF_FOOT_wrench_force (18228, 3) <class 'numpy.ndarray'>
-->    LF_FOOT_wrench_torque (18228, 3) <class 'numpy.ndarray'>
-->    LH_FOOT_contact (18228,) <class 'numpy.ndarray'>
-->    LH_FOOT_friction_coef (18228,) <class 'numpy.ndarray'>
--> 

In [ ]:
print([float(ts) for ts in t[0:5]])
print([float(ts) for ts in aligned["sensors"]["anymal_state_odometry"]["timestamp_50hz"][0:5]])
# first timestamp matched the first the first time stamp from your previous alignment checks. its the timestamp from the first command.

[1727776107.42612, 1727776107.44612, 1727776107.46612, 1727776107.48612, 1727776107.50612]
[1727776107.42612, 1727776107.44612, 1727776107.46612, 1727776107.48612, 1727776107.50612]


# Dataset buildout

In [ ]:
# Observations 
# Base Linear velocity --> anymal_state_state_estimator["twist_lin"]
# Base Angular Velocity --> anymal_state_state_estimator["twist_ang"]
# Gravity Vector Measurement --> anymal_imu["orien"] and ["lin_acc"] * need to derive
# Joint Positions --> anymal_state_state_estimator["joint_positions"]
# Joint Velocities --> anymal_state_state_estimator["joint_velocities"]
# Previous Actions --> anymal_state_actuator["XX_command_position"] XX for 0 to 11
# Commanded Base Lin Velocity --> anymal_command_twist["linear"]
# Commanded Base Ang Velocity --> anymal_command_twist["angular"]

# Actions
# anymal_state_actuator["XX_command_position"] XX for 0 to 11

# Terminals
# every 20s --> 20s * 50Hz data = terminal every 1000 steps

In [ ]:
def build_offline_dataset(data, episode_len_s=20, hz=50):
    """Convert aligned ANYmal sensor data into offline RL dataset."""

    est = data["sensors"]["anymal_state_state_estimator"]
    act = data["sensors"]["anymal_state_actuator"]
    cmd = data["sensors"]["anymal_command_twist"]

    base_lin_vel = est["twist_lin"]          # (T, 3)
    base_ang_vel = est["twist_ang"]          # (T, 3)
    joint_pos = est["joint_positions"]       # (T, 12)
    joint_vel = est["joint_velocities"]      # (T, 12)
    cmd_lin = cmd["linear"]                  # (T, 3)
    cmd_ang = cmd["angular"]                 # (T, 3)

    act_keys = [f"{i:02d}_command_position" for i in range(12)]
    actions = np.stack([act[k] for k in act_keys], axis=-1)   # (T, 12)

    obs = np.concatenate([
        base_lin_vel,
        base_ang_vel,
        joint_pos,
        joint_vel,
        actions,         # previous action
        cmd_lin,
        cmd_ang,
    ], axis=-1)  # (T, obs_dim)

    # --- Shift for next_observations ---
    observations = obs[:-1]
    next_observations = obs[1:]
    actions = actions[:-1]

    # Terminals every 20s (20s * 50hz = 1000 steps)
    T = len(observations)
    episode_len = int(episode_len_s * hz)
    terminals = np.zeros(T, dtype=bool)
    terminals[np.arange(episode_len - 1, T, episode_len)] = True

    # offline dataset 
    dataset = dict(
        observations=observations,
        actions=actions,
        next_observations=next_observations,
        terminals=terminals,
    )

    return dataset


In [8]:
dataset = build_offline_dataset(aligned)

In [11]:
print(dataset["observations"].shape, dataset["actions"].shape, dataset["next_observations"].shape ,dataset["terminals"].shape)

(18227, 48) (18227, 12) (18227, 48) (18227,)
